**Date Created**: 04-16-2025  
**Date of the last Update:** 04-16-2025  
___

► **Install scholarly:**  
Install scholarly to retrieve author and publication information from Google Scholar:
```
pip install scholarly
```
   
Documentation: https://scholarly.readthedocs.io/en/stable/quickstart.html  
GitHub: https://github.com/scholarly-python-package/scholarly  

# Imports & Settings

In [ ]:
import json
import re
from datetime import datetime
from pathlib import Path

from scholarly import scholarly

# Getting a list of publications from a profile
**Ruslan's profile link:** https://scholar.google.com/citations?user=U7yVbHIAAAAJ&hl=en  


In [2]:
today = datetime.today().strftime('%m-%d-%Y')
filename = f'publications_detailed_{today}.json'
filepath = Path("/restricted/projectnb/trypmito/personal_workspaces/annaparf/projects/afasilab.github.io/publications/gs_json/")
filepath.mkdir(parents=True, exist_ok=True)
full_path = filepath / filename

In [3]:
author_id = 'U7yVbHIAAAAJ' # Ruslan
search_query = scholarly.search_author_id(author_id)
author = scholarly.fill(search_query, sections=['publications'])

In [4]:
publications = []

for pub in author['publications']:
    filled_pub = scholarly.fill(pub)
    bib = filled_pub.get('bib', {})
    
    publications.append({
        'title': bib.get('title', ''),
        'authors': bib.get('author', ''),
        'year': bib.get('pub_year', ''),
        'journal': bib.get('journal', ''),
        'volume': bib.get('volume', ''),
        'number': bib.get('number', ''),
        'pages': bib.get('pages', ''),
        'publisher': bib.get('publisher', ''),
        'abstract': bib.get('abstract', ''),
        'citation': bib.get('citation', ''),
        'num_citations': filled_pub.get('num_citations', 0),
        'pub_url': filled_pub.get('pub_url', ''),
        'author_pub_id': filled_pub.get('author_pub_id', ''),
    })

with open(full_path, 'w', encoding='utf-8') as f:
    json.dump(publications, f, ensure_ascii=False, indent=2)

# Prepare citates

In [5]:
# Paths
json_path = Path("/restricted/projectnb/trypmito/personal_workspaces/annaparf/projects/afasilab.github.io/publications/gs_json/publications_detailed_04-16-2025.json")
index_path = Path("/restricted/projectnb/trypmito/personal_workspaces/annaparf/projects/afasilab.github.io/index.html")


# Upload json
with open(json_path, 'r', encoding='utf-8') as f:
    publications = json.load(f)


# Delete without year, sort in descending order
publications = [p for p in publications if p.get('year')]
publications.sort(key=lambda x: int(x['year']), reverse=True)


# Generate html block (section)
pub_html_blocks = []
for pub in publications:
    authors = pub.get('authors', '')
    title = pub.get('title', '')
    journal = pub.get('journal', '')
    volume = pub.get('volume', '')
    number = pub.get('number', '')
    pages = pub.get('pages', '')
    year = pub.get('year', '')
    url = pub.get('pub_url', '')

    # Генерация текста цитаты
    citation = f"{authors}. {title}."
    citation += f" <em>{journal}</em>" if journal else ""
    if volume:
        citation += f", <strong>{volume}</strong>"
    if number:
        citation += f"({number})"
    if pages:
        citation += f", {pages}"
    citation += f", {year}."

    entry_html = f"""
    <div class="pub-entry">
      <p>{citation}<br><em>(Link to this publication: <a href="{url}" target="_blank">{url}</a>)</em></p>
    </div>
    """
    pub_html_blocks.append(entry_html)


# Final section with publications
new_publications_block = f"""
<section id="publications">
  <h2>Publications</h2>
  <div class="publications">
    {''.join(pub_html_blocks)}
  </div>
</section>
"""


# Upload index.html
with open(index_path, 'r', encoding='utf-8') as f:
    html = f.read()


# Replacing the publication section
html = re.sub(
    r'<section id="publications">.*?</section>',
    new_publications_block,
    html,
    flags=re.DOTALL
)

In [6]:
# Save the updated index.html
with open(index_path, 'w', encoding='utf-8') as f:
    f.write(html)

print("✅ index.html is successfully updated!")

✅ index.html is successfully updated!
